<a href="https://colab.research.google.com/github/Rajora0/nlp_studies/blob/main/Word2Vec_interpreta%C3%A7%C3%A3o_da_linguagem_humana_com_Word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec: interpretação da linguagem humana com Word embedding

## Montar Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive/')

!ls '/gdrive/MyDrive/NLP_ESTUDOS/Dados/word2vec'

Drive already mounted at /gdrive/; to attempt to forcibly remount, call drive.mount("/gdrive/", force_remount=True).
cbow_s300.txt  skip_s300.txt  teste.csv  treino.csv


## Carregando os dados

In [ ]:
#!pip install -U nltk -qq

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
path = '/gdrive/MyDrive/NLP_ESTUDOS/Dados/word2vec/'

In [ ]:
artigo_treino = pd.read_csv(path+"treino.csv")
artigo_teste = pd.read_csv(path+"teste.csv")

In [ ]:
artigo_treino.head()

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...


In [ ]:
artigo_teste.head()

,title,text,date,category,subcategory,link
0,Grandes irmãos,"RIO DE JANEIRO - O Brasil, cada vez menos famí...",2017-03-06,colunas,ruycastro,http://www1.folha.uol.com.br/colunas/ruycastro...
1,Haddad congela orçamento e suspende emendas de...,"O prefeito de São Paulo, Fernando Haddad (PT),...",2016-08-10,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
2,Proposta de reforma da Fifa tem a divulgação d...,"A Fifa divulgou, nesta quinta (10), um relatór...",2015-10-09,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/09/1...
3,"Mercado incipiente, internet das coisas conect...","Bueiros, coleiras, aparelhos hospitalares, ele...",2016-11-09,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/09/1...
4,"Mortes: Psicanalista, estudou o autismo em cri...",Toda vez que o grupo de amigos de Silvana Rabe...,2017-02-07,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2017/07...


## One-hot encode

- https://arthurlambletvaz.medium.com/one-hot-encoding-o-que-%C3%A9-cd2e8d302ae0

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
texto = [
         "tenha um bom dia",
         "tenha um péssimo dia",
         "tenha um ótimo dia"
]

In [ ]:
vetorizador = CountVectorizer()
vetorizador.fit(texto)

CountVectorizer()

In [ ]:
print(vetorizador.vocabulary_)

{'tenha': 3, 'um': 4, 'bom': 0, 'dia': 1, 'péssimo': 2, 'ótimo': 5}


In [ ]:
vetor_bom = vetorizador.transform(["bom"])
print(vetor_bom.toarray())

[[1 0 0 0 0 0]]


In [ ]:
vetor_otimo = vetorizador.transform(["ótimo"])
print(vetor_otimo.toarray())

[[0 0 0 0 0 1]]


### One-hot enconde em outro contexto


In [ ]:
# Pandas

ids = [11, 22, 33, 44, 55, 66, 77]
countries = ['Spain', 'France', 'Spain', 'Germany', 'France']

df = pd.DataFrame(list(zip(ids, countries)),
                  columns=['Ids', 'Countries'])

df.head()

,Ids,Countries
0,11,Spain
1,22,France
2,33,Spain
3,44,Germany
4,55,France


In [ ]:
y = pd.get_dummies(df.Countries, prefix='Country')
print(y.head())

   Country_France  Country_Germany  Country_Spain
0               0                0              1
1               1                0              0
2               0                0              1
3               0                1              0
4               1                0              0


In [ ]:
# Scikit-Learn

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
y = LabelBinarizer().fit_transform(df.Countries)
y

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0]])

In [ ]:
x = [[11, "Spain"], [22, "France"], [33, "Spain"], [44, "Germany"], [55, "France"]]
y = OneHotEncoder().fit_transform(x).toarray()
print(y)

[[1. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0.]]


## Word2Vec



### Artigos

- https://medium.com/@everton.tomalok/word2vec-e-sua-import%C3%A2ncia-na-etapa-de-pr%C3%A9-processamento-d0813acfc8ab
- http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc

- https://arxiv.org/pdf/1301.3781.pdf

### Exemplo

In [ ]:
from gensim.models import KeyedVectors


In [ ]:
with open(path+'cbow_s300.txt') as r:
  for lista in range(5):
    print(next(r))

929606 300

</s> -0.001667 -0.000158 -0.000026 0.001300 -0.000796 0.001527 0.000046 0.000584 0.000449 -0.000100 0.000353 0.001251 0.001069 0.000506 0.000574 0.000838 -0.000930 -0.001220 0.000317 0.001315 -0.001120 0.001373 -0.000040 -0.001580 0.000421 -0.000667 -0.001556 -0.000746 0.001604 0.001157 -0.000027 0.000354 0.000358 -0.000527 -0.000573 -0.001512 -0.001557 -0.001637 0.001617 -0.001511 -0.001022 -0.001426 0.001086 -0.001033 0.000593 0.000724 0.000627 -0.000450 -0.001140 0.000333 0.000524 0.001541 0.000284 0.000617 -0.000807 -0.000088 -0.000364 0.001126 -0.001230 -0.001138 -0.001280 0.001330 0.001257 0.000576 0.000764 0.000684 0.001008 -0.000215 -0.000629 -0.001228 -0.001557 -0.000311 -0.000246 0.000045 0.001136 -0.000645 -0.000549 0.001099 0.000858 -0.000886 0.000553 0.000303 0.001433 0.000732 0.001321 -0.000894 -0.000700 -0.000661 -0.001484 -0.000950 -0.001556 -0.000809 0.000348 -0.000068 0.000724 -0.000569 -0.000161 -0.001628 -0.001437 -0.000259 -0.000296 -0.001571 0.000149 0

In [ ]:
modelo = KeyedVectors.load_word2vec_format(path+'cbow_s300.txt')


In [ ]:
modelo.get_vector("china")

array([-1.49033e-01,  1.26020e-01,  2.17628e-01,  1.82684e-01,
        1.65151e-01, -1.59660e-01, -2.34411e-01,  6.00570e-02,
        8.03680e-02,  2.87578e-01, -4.81100e-03, -5.68800e-02,
        2.15676e-01,  8.65540e-02,  1.25983e-01,  3.36157e-01,
       -1.83254e-01, -1.18499e-01,  1.13010e-02,  1.03814e-01,
        9.37640e-02,  2.90178e-01, -1.64395e-01, -1.13300e-02,
       -1.80676e-01, -1.15820e-02,  1.08728e-01,  1.65898e-01,
        9.37900e-02,  2.66767e-01, -1.29890e-02,  9.16030e-02,
        2.21292e-01, -1.36497e-01, -4.26350e-02, -1.30038e-01,
        2.17067e-01, -1.01963e-01, -3.70960e-02,  1.42155e-01,
        3.41109e-01,  2.46560e-01,  1.27458e-01,  5.72360e-02,
       -1.47962e-01, -1.60290e-02,  1.86533e-01,  7.71550e-02,
       -3.50024e-01, -4.06085e-01,  1.67131e-01, -4.75230e-02,
        5.13780e-02, -1.28224e-01,  1.06580e-02, -2.92652e-01,
        1.40540e-01, -4.57049e-01,  1.31094e-01,  2.03234e-01,
        2.94019e-01,  7.38370e-02,  1.11554e-01, -1.642

In [ ]:
modelo.most_similar("china")

[('rússia', 0.7320704460144043),
 ('índia', 0.7241617441177368),
 ('tailândia', 0.701935887336731),
 ('indonésia', 0.6860769987106323),
 ('turquia', 0.6741335988044739),
 ('malásia', 0.6665689945220947),
 ('mongólia', 0.6593616008758545),
 ('manchúria', 0.6581847667694092),
 ('urss', 0.6581669449806213),
 ('grã-bretanha', 0.6568098068237305)]

In [ ]:
modelo.most_similar("itália")

[('frança', 0.7983609437942505),
 ('grécia', 0.7547340989112854),
 ('espanha', 0.7417078018188477),
 ('holanda', 0.7343162298202515),
 ('grã-bretanha', 0.7281740307807922),
 ('inglaterra', 0.7172061204910278),
 ('bélgica', 0.7155542373657227),
 ('alemanha', 0.7135661840438843),
 ('suécia', 0.6831519603729248),
 ('áustria', 0.6822685599327087)]

In [ ]:
modelo.most_similar(positive=["brasil","argentina"])

[('chile', 0.6781662702560425),
 ('peru', 0.6348033547401428),
 ('venezuela', 0.6273865103721619),
 ('equador', 0.6037014722824097),
 ('bolívia', 0.6017140746116638),
 ('haiti', 0.5993807315826416),
 ('méxico', 0.5962306261062622),
 ('paraguai', 0.5957703590393066),
 ('uruguai', 0.5903672575950623),
 ('japão', 0.5893509984016418)]

In [ ]:
#nuvens -> nuvem : estrelas -> estrela 
#nuvens + estrela - nuvem = estrelas

modelo.most_similar(positive=["nuvens","estrela"],negative=['nuvem'])

[('estrelas', 0.5497429966926575),
 ('plêiades', 0.379197895526886),
 ('colinas', 0.3746805191040039),
 ('trovoadas', 0.373703271150589),
 ('sombras', 0.3734194040298462),
 ('pombas', 0.3726757764816284),
 ('corredoras', 0.3640727698802948),
 ('cigarras', 0.36065393686294556),
 ('galáxias', 0.35754913091659546),
 ('luas', 0.3575345277786255)]

## Vetorizando os dados

In [ ]:
import nltk 
import string 

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def tokenizador(texto): 
  texto = texto.lower() 
  lista_alfanumerico = [] 
  for token_valido in nltk.word_tokenize(texto):

    if token_valido in string.punctuation: continue
    else: lista_alfanumerico.append(token_valido)

  return lista_alfanumerico

tokenizador('Texto exemplo, 1234.')

['texto', 'exemplo', '1234']

In [ ]:
def combinacao_de_vet_soma(modelo,palavras_numeros): 

  vetor_resultante = np.zeros(300) 

  for pn in palavras_numeros: 

    try:

      vetor_resultante += modelo.get_vector(pn)

    except KeyError:
      if pn.isnumeric():
        vetor_resultante += modelo.get_vector('0'*len(pn))
      else:
        vetor_resultante += modelo.get_vector('unknown')
  
  return vetor_resultante

In [ ]:
palavras_numeros = tokenizador('Texto caelunx')
vetor_texto = combinacao_de_vet_soma(modelo,palavras_numeros)

In [ ]:
print(vetor_texto.shape,vetor_texto)

(300,) [ 4.04902007e-01  1.61033005e-01 -1.28350001e-01  4.63957988e-01
 -3.79070006e-02  9.79099981e-02  6.06219992e-02  2.50669867e-02
 -1.45315003e-01 -1.20187001e-01 -1.45029984e-02 -3.97820026e-02
 -5.53439990e-01  3.39491010e-01  4.74720001e-02  2.32610006e-01
 -7.91507989e-01  7.62100518e-03 -4.30413000e-01 -1.62210986e-01
 -1.36736006e-01 -2.92862996e-01 -7.81050026e-02  4.56106991e-01
 -1.08597010e-01 -1.72903001e-01 -3.94289017e-01 -7.94299692e-03
  1.54560007e-01 -1.81829929e-02 -1.44808993e-01 -1.92361996e-01
  5.27470000e-02  2.77960012e-02  3.75599414e-03  2.15162002e-01
  2.41512008e-01  2.07674988e-01  1.77449007e-01  2.00845003e-01
  1.08239995e-02 -1.03557003e-01  4.52470034e-02  3.54460012e-02
  2.74897987e-01 -7.71570001e-02  4.26173002e-01  4.57773015e-01
  2.98846003e-01  2.85166003e-01  1.75556996e-01  1.93372004e-01
 -2.64183000e-01  1.87287997e-01  2.00636007e-01 -1.48244001e-01
  1.16369996e-01  5.01876995e-01 -2.73249969e-02 -3.36082000e-01
 -1.06219009e-01  

-  https://www.kaggle.com/c/quora-insincere-questions-classification/discussion/71778

In [ ]:
def matriz_vetores(modelo,textos):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vet_soma(modelo,palavras_numeros)

    return matriz

matriz_vetores_treino = matriz_vetores(modelo,artigo_treino.title)
matriz_vetores_teste = matriz_vetores(modelo,artigo_teste.title)
print(matriz_vetores_treino.shape)
print(matriz_vetores_teste.shape) 

(90000, 300)
(20513, 300)


## Regressão Logistica

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
LR = LogisticRegression(max_iter = 125)
LR.fit(matriz_vetores_treino, artigo_treino.category)

LogisticRegression(max_iter=125)

In [ ]:
LR.score(matriz_vetores_teste, artigo_teste.category)

0.7964705308828548

In [ ]:
artigo_teste.category.value_counts()

colunas      6103
mercado      5867
esporte      4663
mundo        2051
cotidiano    1698
ilustrada     131
Name: category, dtype: int64

In [ ]:
artigo_treino.category.value_counts()

mundo        15000
cotidiano    15000
mercado      15000
esporte      15000
ilustrada    15000
colunas      15000
Name: category, dtype: int64

In [ ]:
label_prevista = LR.predict(matriz_vetores_teste)
CR = classification_report(artigo_teste.category, label_prevista)
print(CR)

              precision    recall  f1-score   support

     colunas       0.86      0.72      0.78      6103
   cotidiano       0.61      0.79      0.69      1698
     esporte       0.93      0.88      0.90      4663
   ilustrada       0.13      0.88      0.23       131
     mercado       0.84      0.79      0.81      5867
       mundo       0.74      0.86      0.79      2051

    accuracy                           0.80     20513
   macro avg       0.68      0.82      0.70     20513
weighted avg       0.83      0.80      0.81     20513



In [ ]:
artigo_teste.category[0]

'colunas'

In [ ]:
LR.predict([matriz_vetores_teste[0]])

array(['colunas'], dtype=object)

- https://pt.wikiversity.org/wiki/Observat%C3%B3rio_de_dados/Precis%C3%A3o_e_revoga%C3%A7%C3%A3o

## Dummy Classifier

In [ ]:
from sklearn.dummy import DummyClassifier

In [ ]:
DC = DummyClassifier()
DC.fit(matriz_vetores_treino, artigo_treino.category)

label_prevista = DC.predict(matriz_vetores_teste)
CR = classification_report(artigo_teste.category, label_prevista)
print(CR)

              precision    recall  f1-score   support

     colunas       0.30      1.00      0.46      6103
   cotidiano       0.00      0.00      0.00      1698
     esporte       0.00      0.00      0.00      4663
   ilustrada       0.00      0.00      0.00       131
     mercado       0.00      0.00      0.00      5867
       mundo       0.00      0.00      0.00      2051

    accuracy                           0.30     20513
   macro avg       0.05      0.17      0.08     20513
weighted avg       0.09      0.30      0.14     20513



## Skipgram

In [ ]:
modelo_skipgram = KeyedVectors.load_word2vec_format(path+'skip_s300.txt')

In [ ]:
matriz_vetores_treino_skipgram = matriz_vetores(modelo_skipgram,artigo_treino.title)
matriz_vetores_teste_skipgram  = matriz_vetores(modelo_skipgram,artigo_teste.title)
print(matriz_vetores_treino_skipgram.shape)
print(matriz_vetores_teste_skipgram.shape) 

(90000, 300)
(20513, 300)


In [ ]:
LR_skipgram = LogisticRegression()
LR_skipgram.fit(matriz_vetores_treino_skipgram, artigo_treino.category)

label_prevista = LR_skipgram.predict(matriz_vetores_teste_skipgram)
CR = classification_report(artigo_teste.category, label_prevista)
print(CR)

              precision    recall  f1-score   support

     colunas       0.86      0.72      0.78      6103
   cotidiano       0.62      0.80      0.70      1698
     esporte       0.93      0.89      0.91      4663
   ilustrada       0.15      0.92      0.26       131
     mercado       0.85      0.81      0.83      5867
       mundo       0.76      0.86      0.81      2051

    accuracy                           0.81     20513
   macro avg       0.69      0.83      0.71     20513
weighted avg       0.84      0.81      0.82     20513



In [ ]:
DC_skipgram = DummyClassifier()
DC_skipgram.fit(matriz_vetores_treino_skipgram, artigo_treino.category)

label_prevista = DC_skipgram.predict(matriz_vetores_teste_skipgram)
CR = classification_report(artigo_teste.category, label_prevista)
print(CR)

              precision    recall  f1-score   support

     colunas       0.30      1.00      0.46      6103
   cotidiano       0.00      0.00      0.00      1698
     esporte       0.00      0.00      0.00      4663
   ilustrada       0.00      0.00      0.00       131
     mercado       0.00      0.00      0.00      5867
       mundo       0.00      0.00      0.00      2051

    accuracy                           0.30     20513
   macro avg       0.05      0.17      0.08     20513
weighted avg       0.09      0.30      0.14     20513

